# Julia Set Visualization Notebook

## Introduction

This Jupyter notebook is dedicated to the computation and visualization of Julia Sets. Julia Sets are complex fractals generated from iterative functions and are renowned for their strikingly intricate patterns. This notebook serves as an interactive tool to explore these fascinating constructs from the realm of complex dynamics and fractal geometry.

## Notebook Structure

- **CODE Cell 1**: Importing required libraries and setting up the environment.
- **CODE Cell 2**: Function to compute the Julia Set.
- **CODE Cell 3**: Function for plotting the Julia Set.
- **CODE Cell 4**: Interactive widgets for user input.
- **CODE Cell 5**: Event handling and display of the Julia Set.

## Dependencies

In [3]:
%%capture
!pip install numpy
!pip install matplotlib
!pip install numba
!pip install ipywidgets

# Required Libraries and Setup
Import all necessary libraries for numerical computations, visualization, and interactive widgets. Also, set up the directory for caching, if required for performance optimization.

# imports

In [21]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit, prange
from ipywidgets import FloatSlider, IntSlider, Text, Button, Output, VBox, HBox, Label

# Compute the Julia Set
Define the `compute_fractal_parallel` function to calculate the Julia Set for a given complex number. This function employs parallel computing for efficient processing.

In [22]:
@jit(nopython=True)
def julia_set(width, height, x_min, x_max, y_min, y_max, c, max_iter):
    x = np.linspace(x_min, x_max, width)
    y = np.linspace(y_min, y_max, height)
    fractal = np.zeros((height, width), dtype=np.int64)

    for i in prange(height):
        for j in prange(width):
            zx = x[j]
            zy = y[i]
            iteration = 0
            while zx*zx + zy*zy < 4 and iteration < max_iter:
                new_zx = zx*zx - zy*zy + c.real
                zy = 2*zx*zy + c.imag
                zx = new_zx
                iteration += 1
            fractal[i, j] = iteration

    return fractal

# Julia Set Visualization
Define a function to plot the Julia Set. This function will take the output from the computation and render it as a visual fractal pattern.

In [23]:
def plot_julia(fractal):
    plt.figure(figsize=(10, 10))
    # Convert the fractal array to float and ensure it's in a format compatible with Matplotlib
    plt.imshow(fractal.astype(float), cmap='hot', extent=[x_min.value, x_max.value, y_min.value, y_max.value])
    plt.colorbar()
    plt.title("Julia Set")
    plt.show()

# User Input Widgets
Set up interactive widgets to allow users to input parameters for generating the Julia Set. These include sliders for setting the complex number and other parameters.

In [24]:
# Define the parameters for the Julia set
c_real = FloatSlider(min=-2.0, max=2.0, step=0.01, value=-0.8, description='Re(c):')
c_imag = FloatSlider(min=-2.0, max=2.0, step=0.01, value=0.156, description='Im(c):')
x_min = FloatSlider(min=-2.0, max=2.0, step=0.01, value=-1.5, description='x_min:')
x_max = FloatSlider(min=-2.0, max=2.0, step=0.01, value=1.5, description='x_max:')
y_min = FloatSlider(min=-2.0, max=2.0, step=0.01, value=-1.5, description='y_min:')
y_max = FloatSlider(min=-2.0, max=2.0, step=0.01, value=1.5, description='y_max:')
max_iter = IntSlider(min=1, max=1000, step=1, value=300, description='Max Iter:')

# Event Handler and Visualization
Create a button event handler to generate and display the Julia Set based on user inputs. This cell ties together the computation and visualization functionalities, allowing users to interactively explore different Julia Sets.

In [27]:
def on_generate_button_clicked(b):
    with output_area:
        output_area.clear_output(wait=True)

        # Extract parameters
        real_part = c_real.value
        imag_part = c_imag.value
        x_min_val = x_min.value
        x_max_val = x_max.value
        y_min_val = y_min.value
        y_max_val = y_max.value
        iterations = max_iter.value

        # Generate the fractal
        fractal = julia_set(800, 800, x_min_val, x_max_val, y_min_val, y_max_val, complex(real_part, imag_part), iterations)
        plot_julia(fractal)

        # Iteration Depth Plot
        plt.figure(figsize=(10, 5))
        plt.imshow(fractal, extent=[x_min_val, x_max_val, y_min_val, y_max_val], cmap='inferno')
        plt.colorbar(label='Iterations to Escape')
        plt.title('Iteration Depth Plot')
        plt.show()

        # Additional Information and Plots
        print(f"Julia Set Parameters:")
        print(f"  - Complex constant (c): {real_part} + {imag_part}i")
        print(f"  - X-axis range: {x_min_val} to {x_max_val}")
        print(f"  - Y-axis range: {y_min_val} to {y_max_val}")
        print(f"  - Maximum Iterations: {iterations}")

        print("\nParameter Insights:")
        print("  - The complex constant 'c' determines the shape and complexity of the Julia set.")
        print("  - The X and Y axis ranges define the region of the complex plane visualized.")
        print("  - The number of iterations affects the detail and depth of the fractal boundary.")

        # Add more plots here as needed

generate_button.on_click(on_generate_button_clicked)

# Display the widgets and the output area
display(VBox([
    HBox([c_real, c_imag]),
    HBox([x_min, x_max]),
    HBox([y_min, y_max]),
    max_iter,
    generate_button,
    output_area
]))